<a href="https://colab.research.google.com/github/pompymandislian/scratch_transformer_model/blob/main/Transformer_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, Dataset
import pytorch_lightning as L # simplify code (reduced epoch and using one more GPU)

In [2]:
# !pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 24.0 MB/s eta 0:00:00


# Tokenize
---

In [4]:
token_to_id = {
              'what': 0,
               'is': 1,
               'monkey': 2,
               'awesome':3,
               '<EOS>' : 4 # sort dict
               }

id_to_token = dict(map(reversed, token_to_id.items())) # dict

id_to_token

{0: 'what', 1: 'is', 2: 'monkey', 3: 'awesome', 4: '<EOS>'}

In [5]:
# inputs
inputs_1 = torch.tensor([[token_to_id['what'],
                        token_to_id['is'],
                        token_to_id['monkey'],
                        token_to_id['<EOS>'], # sort dict
                        token_to_id['awesome']
                        ]])

print('Input 1', inputs_1)

inputs_2 = torch.tensor([[token_to_id['is'],
                        token_to_id['monkey'],
                        token_to_id['<EOS>'],# sort dict
                        token_to_id['awesome'],
                        token_to_id['<EOS>']# sort dict
                        ]])

print('Input 2', inputs_2)

Input 1 tensor([[0, 1, 2, 4, 3]])
Input 2 tensor([[1, 2, 4, 3, 4]])


In [6]:
# example
inputs = torch.tensor([
                      [token_to_id['what'],
                      token_to_id['is'],
                      token_to_id['monkey'],
                      token_to_id['<EOS>'],  # sort dict
                      token_to_id['awesome']],

                      [token_to_id['monkey'],
                      token_to_id['is'],
                      token_to_id['what'],
                      token_to_id['<EOS>'],  # sort dict
                      token_to_id['awesome']]
                  ])
print('Inputs', inputs)

labels = torch.tensor([
                      [token_to_id['is'],
                      token_to_id['monkey'],
                      token_to_id['<EOS>'],
                      token_to_id['awesome'],  # sort dict
                      token_to_id['<EOS>']],

                      [token_to_id['is'],
                      token_to_id['what'],
                      token_to_id['<EOS>'],
                      token_to_id['monkey'],  # sort dict
                      token_to_id['<EOS>']]
                  ])
print('Labels', labels)

dataset = TensorDataset(inputs, labels) # combine label and classification
dataset

Inputs tensor([[0, 1, 2, 4, 3],
        [2, 1, 0, 4, 3]])
Labels tensor([[1, 2, 4, 3, 4],
        [1, 0, 4, 2, 4]])


# Word Embedding
---
*example*

- data input = ["The", "quick", "brown", "fox"]

Embedding:
- "The"  → [0.12, 0.45, -0.23, 0.78]
- "quick" → [0.56, -0.33, 0.75, -0.21]
- "brown" → [0.33, 0.11, 0.22, 0.44]
- "fox" → [0.45, -0.12, 0.67, -0.67]

Position Encoding
- Posisi 0 (The) → [0.01, 0.02, 0.03, 0.04]
- Posisi 1 (quick) → [0.02, -0.01, 0.04, -0.05]
- Posisi 2 (brown) → [0.05, 0.03, -0.02, 0.01]
- Posisi 3 (fox) → [-0.01, 0.04, 0.01, -0.03]


In [13]:
class PositionEncoding(nn.Module):
    def __init__(self, d_model=2, max_len=6):
        super(PositionEncoding, self).__init__()

        # Membuat tensor kosong dengan ukuran (max_len, d_model) untuk menyimpan posisi encoding
        pe = torch.zeros(max_len, d_model)

        # Membuat tensor 'position' yang berisi posisi setiap token dalam urutan, dimulai dari 0 hingga max_len-1
        # shape = (max_len, 1)
        position = torch.arange(start=0, end=max_len, dtype=torch.float).unsqueeze(dim=1)

        # Membuat tensor 'embedding_index' yang berisi indeks dimensi untuk posisi encoding (0, 2, 4, ...)
        # shape = (d_model // 2,)
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()

        # Membuat faktor pembagi untuk posisi encoding menggunakan rumus sinusoidal
        # Rumus ini akan menghasilkan pembagian berdasarkan 10000^(2k/d_model) di setiap dimensi
        div_term = 1 / torch.tensor(10000.0) ** (embedding_index / d_model)

        # Menghitung nilai sine untuk dimensi yang berpasangan (dimensi genap)
        # dan cosine untuk dimensi yang ganjil
        # pe[:, 0::2] memilih kolom genap (dimensi 0, 2, 4, ...) dan memberi nilai sinus
        pe[:, 0::2] = torch.sin(position * div_term)

        # pe[:, 1::2] memilih kolom ganjil (dimensi 1, 3, 5, ...) dan memberi nilai cosinus
        pe[:, 1::2] = torch.cos(position * div_term)

        # Menyimpan 'pe' sebagai buffer agar tidak dioptimasi selama pelatihan
        # 'pe' berisi posisi encoding yang dihitung di atas kirim to GPU
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Menambahkan posisi encoding ke input x yang sudah ada (biasanya embedding token)
        return x + self.pe[:x.size(0), :]

# Masked Self-Attention
---

EX: "The"

Matriks bobot yang dipelajari untuk Query, Key, dan Value:
- W_Q → Matriks yang digunakan untuk menghitung Q
- W_K → Matriks yang digunakan untuk menghitung K
- W_V → Matriks yang digunakan untuk menghitung V

- Q = [0.12, 0.22, 0.32, 0.42] × W_Q → Hasilnya adalah vektor Q untuk token "The".
- K = [0.12, 0.22, 0.32, 0.42] × W_K → Hasilnya adalah vektor K untuk token "The".
- V = [0.12, 0.22, 0.32, 0.42] × W_V → Hasilnya adalah vektor V untuk token "The".

Penjelasan:
- Query (Q) mencari informasi relevan di dalam input. Misalnya, jika kita memproses kata "The", kita ingin tahu kata mana yang relevan dengan "The".
- Key (K) adalah informasi yang disediakan oleh token lain dalam urutan, yang akan dibandingkan dengan Query.
- Value (V) adalah informasi yang akan digunakan untuk memperbarui representasi token, berdasarkan seberapa relevan Key tersebut dengan Query.

In [28]:
class Attention(nn.Module):
    def __init__(self, d_model=2):
        super().__init__()
        # Matriks bobot linear untuk Query (Q), Key (K), dan Value (V)
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = 0  # Untuk baris
        self.col_dim = 1  # Untuk kolom

    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
        # Menghitung Q, K, dan V dengan mentransformasikan encoding input menggunakan linear layers
        Q = self.W_q(encodings_for_q)  # Transformasi untuk Query
        K = self.W_k(encodings_for_k)  # Transformasi untuk Key
        V = self.W_v(encodings_for_v)  # Transformasi untuk Value

        # Menghitung kesamaan (similarity) antara Query (Q) dan Key (K)
        # Pastikan Q memiliki bentuk (batch_size, seq_len, d_model)
        # K harus ditranspose menjadi (batch_size, d_model, seq_len) untuk perkalian matriks
        sims = torch.matmul(Q, K.transpose(-2, -1))  # transpose K agar cocok dengan Q

        # Scaling similarity berdasarkan dimensi Key (K)
        scaled_sims = sims / torch.sqrt(torch.tensor(K.size(self.col_dim), dtype=torch.float32))  # Skala untuk stabilitas

        # Apakah ada mask? Mask digunakan untuk mengabaikan beberapa posisi dalam perhatian
        if mask is not None:
            # Menggunakan mask untuk menutupi (masking) beberapa nilai dalam similarity
            # Misalnya, untuk padding atau posisi yang tidak diinginkan, kita set hasil similarity-nya ke nilai sangat besar
            scaled_sims = scaled_sims.masked_fill(mask, value=1e9)

        # Menghitung skor perhatian dengan Softmax untuk normalisasi
        attention_percentages = F.softmax(scaled_sims, dim=-1)  # Softmax pada dimensi terakhir (seq_len)

        # Menghitung hasil perhatian dengan mengalikan skor perhatian dengan V (Value)
        attention_score = torch.matmul(attention_percentages, V)

        return attention_score

# Combine all class
---

In [35]:
class DecoderOnlyTransformer(L.LightningModule):
    def __init__(self, num_tokens=4, d_model=2, max_len=6):
        super().__init__()

        # Step 1: Embedding layer
        self.embedding = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)

        # Step 2: Positional encoding
        self.position_encoding = PositionEncoding(d_model=d_model, max_len=max_len)

        # Step 3: Attention layer
        self.attention = Attention(d_model=d_model)

        # Step 4: Fully connected layer
        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

        # Step 5: Loss function
        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_id):
        # Step 1: Embedding the input tokens
        word_embeddings = self.embedding(token_id)

        # Step 2: Add positional encoding to the embeddings
        # Slice the positional encoding based on the sequence length of the input (word_embeddings.size(1))
        position_encoded = word_embeddings + self.position_encoding.pe[:word_embeddings.size(1), :]

        # Step 3: Create mask (upper triangular matrix for look-ahead masking)
        seq_length = token_id.size(1)  # panjang urutan
        mask = torch.triu(torch.ones((seq_length, seq_length)), diagonal=1)
        mask = mask == 1  # Membuat mask dengan nilai 1 di atas diagonal (untuk "look-ahead" masking)

        # Step 4: Perform self-attention
        self_attention_values = self.attention(position_encoded, position_encoded, position_encoded, mask)

        # Step 5: Add residual connection
        residual_connection_values = position_encoded + self_attention_values

        # Step 6: Apply the final fully connected layer
        fc_layer_output = self.fc_layer(residual_connection_values)

        return fc_layer_output

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)

    def training_step(self, batch, batch_index):
        input_tokens, label = batch
        output = self.forward(input_tokens)
        loss = self.loss(output.view(-1, output.size(-1)), label.view(-1))
        return loss

# Training and Generated data
---

In [36]:
# arsitekture model
model = DecoderOnlyTransformer(num_tokens=len(token_to_id),
                               d_model=2, max_len=6)
model

DecoderOnlyTransformer(
  (embedding): Embedding(5, 2)
  (position_encoding): PositionEncoding()
  (attention): Attention(
    (W_q): Linear(in_features=2, out_features=2, bias=False)
    (W_k): Linear(in_features=2, out_features=2, bias=False)
    (W_v): Linear(in_features=2, out_features=2, bias=False)
  )
  (fc_layer): Linear(in_features=2, out_features=5, bias=True)
  (loss): CrossEntropyLoss()
)

In [49]:
# Membuat dataset yang lebih beragam dengan kalimat yang lebih kompleks
class SimpleDataset(Dataset):
    def __init__(self, num_samples, seq_length, num_tokens, token_to_id):
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.num_tokens = num_tokens
        self.token_to_id = token_to_id

        # Contoh kalimat yang lebih kompleks
        sentences = [
            ['what', 'is', 'monkey', '<EOS>'],
            ['monkey', 'is','monkey', '<EOS>'],
            ['what', 'is', 'awesome', '<EOS>'],
            ['what', 'monkey', 'awesome', '<EOS>'],
            ['monkey', 'is', 'awesome', '<EOS>'],
            ['awesome', 'is', 'monkey', '<EOS>']
        ]

        self.data = []
        self.labels = []

        for _ in range(num_samples):
            sentence = sentences[_ % len(sentences)]  # Rotasi kalimat untuk variasi
            input_tokens = [self.token_to_id[word] for word in sentence]
            target_tokens = input_tokens[1:] + [self.token_to_id['<EOS>']]  # Label adalah input shift ke kanan

            self.data.append(input_tokens)
            self.labels.append(target_tokens)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor(self.labels[idx])

# Menggunakan dataset baru yang lebih beragam
train_dataset = SimpleDataset(num_samples=1000, seq_length=5, num_tokens=len(token_to_id), token_to_id=token_to_id)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Melihat hasil dataset
for batch_data, batch_labels in train_dataloader:
    print('Batch Data:', batch_data)
    print('Batch Labels:', batch_labels)

Batch Data: tensor([[0, 1, 2, 4],
        [3, 1, 2, 4],
        [0, 1, 3, 4],
        [2, 1, 2, 4],
        [0, 1, 2, 4],
        [3, 1, 2, 4],
        [3, 1, 2, 4],
        [0, 1, 3, 4],
        [3, 1, 2, 4],
        [0, 1, 3, 4],
        [2, 1, 2, 4],
        [2, 1, 3, 4],
        [3, 1, 2, 4],
        [3, 1, 2, 4],
        [2, 1, 3, 4],
        [0, 1, 3, 4],
        [3, 1, 2, 4],
        [0, 1, 3, 4],
        [2, 1, 3, 4],
        [3, 1, 2, 4],
        [2, 1, 2, 4],
        [0, 2, 3, 4],
        [0, 1, 2, 4],
        [0, 1, 2, 4],
        [2, 1, 3, 4],
        [2, 1, 3, 4],
        [2, 1, 3, 4],
        [0, 1, 2, 4],
        [0, 2, 3, 4],
        [2, 1, 3, 4],
        [2, 1, 3, 4],
        [0, 1, 2, 4]])
Batch Labels: tensor([[1, 2, 4, 4],
        [1, 2, 4, 4],
        [1, 3, 4, 4],
        [1, 2, 4, 4],
        [1, 2, 4, 4],
        [1, 2, 4, 4],
        [1, 2, 4, 4],
        [1, 3, 4, 4],
        [1, 2, 4, 4],
        [1, 3, 4, 4],
        [1, 2, 4, 4],
        [1, 3, 4, 4],
     

In [57]:
# Setup Trainer with TensorBoard logger
from pytorch_lightning.loggers import TensorBoardLogger

# Set up TensorBoard logger
logger = TensorBoardLogger("tb_logs", name="model")

# Setup Trainer with logger
trainer = L.Trainer(max_epochs=30, logger=logger)

# Assuming train_dataloader is defined elsewhere, start training
trainer.fit(model, train_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type             | Params | Mode 
---------------------------------------------------------------
0 | embedding         | Embedding        | 10     | train
1 | position_encoding | PositionEncoding | 0      | train
2 | attention         | Attention        | 12     | train
3 | fc_layer          | Linear           | 15     | train
4 | loss              | CrossEntropyLoss | 0      | train
---------------------------------------------------------------
37        Trainable params
0         Non-trainable params
37        Total params
0.000     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


# Predict
---

In [56]:
# Inisialisasi input
model_input = torch.tensor([token_to_id['what'],
                            token_to_id['monkey'],
                            token_to_id['is'],
                            token_to_id['<EOS>']])

model_input = model_input.unsqueeze(0)  # Menambahkan dimensi batch (shape: [1, seq_len])

# Tentukan panjang input
input_length = model_input.size(dim=1)

# Prediksi pertama
predictions = model(model_input)  # Mendapatkan output dari model
predicted_id = torch.argmax(predictions[:, -1, :], dim=-1)  # Mengambil prediksi untuk token terakhir
predicted_ids = predicted_id.unsqueeze(0)  # Membuatnya menjadi tensor dengan dimensi yang benar

# Tentukan panjang maksimal output
max_length = 6

# Mulai proses prediksi bertahap
for _ in range(input_length, max_length):
    if predicted_id == token_to_id['what']:
        break

    # Gabungkan prediksi baru ke dalam input model
    model_input = torch.cat([model_input, predicted_ids[:, -1].unsqueeze(1)], dim=1)  # Menambahkan predicted_id ke input

    # Dapatkan prediksi untuk token berikutnya
    predictions = model(model_input)  # Mendapatkan output dari model
    predicted_id = torch.argmax(predictions[:, -1, :], dim=-1)  # Mengambil token dengan skor tertinggi untuk posisi berikutnya

    # Menambahkan predicted_id ke predicted_ids
    predicted_ids = torch.cat([predicted_ids, predicted_id.unsqueeze(0)], dim=1)

# Menampilkan hasil prediksi
print("Predicted Tokens:")
for id in predicted_ids[0]:  # Mengambil urutan token
    print('\t', id_to_token[id.item()])

Predicted Tokens:
	 <EOS>


# Full Class Transformer Model
---

In [ ]:
class PositionEncoding(nn.Module):
    def __init__(self, d_model=2, max_len=6):
        super(PositionEncoding, self).__init__()

        # Membuat tensor kosong dengan ukuran (max_len, d_model) untuk menyimpan posisi encoding
        pe = torch.zeros(max_len, d_model)

        # Membuat tensor 'position' yang berisi posisi setiap token dalam urutan, dimulai dari 0 hingga max_len-1
        # shape = (max_len, 1)
        position = torch.arange(start=0, end=max_len, dtype=torch.float).unsqueeze(dim=1)

        # Membuat tensor 'embedding_index' yang berisi indeks dimensi untuk posisi encoding (0, 2, 4, ...)
        # shape = (d_model // 2,)
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()

        # Membuat faktor pembagi untuk posisi encoding menggunakan rumus sinusoidal
        # Rumus ini akan menghasilkan pembagian berdasarkan 10000^(2k/d_model) di setiap dimensi
        div_term = 1 / torch.tensor(10000.0) ** (embedding_index / d_model)

        # Menghitung nilai sine untuk dimensi yang berpasangan (dimensi genap)
        # dan cosine untuk dimensi yang ganjil
        # pe[:, 0::2] memilih kolom genap (dimensi 0, 2, 4, ...) dan memberi nilai sinus
        pe[:, 0::2] = torch.sin(position * div_term)

        # pe[:, 1::2] memilih kolom ganjil (dimensi 1, 3, 5, ...) dan memberi nilai cosinus
        pe[:, 1::2] = torch.cos(position * div_term)

        # Menyimpan 'pe' sebagai buffer agar tidak dioptimasi selama pelatihan
        # 'pe' berisi posisi encoding yang dihitung di atas kirim to GPU
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Menambahkan posisi encoding ke input x yang sudah ada (biasanya embedding token)
        return x + self.pe[:x.size(0), :]

class Attention(nn.Module):
    def __init__(self, d_model=2):
        super().__init__()
        # Matriks bobot linear untuk Query (Q), Key (K), dan Value (V)
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = 0  # Untuk baris
        self.col_dim = 1  # Untuk kolom

    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
        # Menghitung Q, K, dan V dengan mentransformasikan encoding input menggunakan linear layers
        Q = self.W_q(encodings_for_q)  # Transformasi untuk Query
        K = self.W_k(encodings_for_k)  # Transformasi untuk Key
        V = self.W_v(encodings_for_v)  # Transformasi untuk Value

        # Menghitung kesamaan (similarity) antara Query (Q) dan Key (K)
        # Pastikan Q memiliki bentuk (batch_size, seq_len, d_model)
        # K harus ditranspose menjadi (batch_size, d_model, seq_len) untuk perkalian matriks
        sims = torch.matmul(Q, K.transpose(-2, -1))  # transpose K agar cocok dengan Q

        # Scaling similarity berdasarkan dimensi Key (K)
        scaled_sims = sims / torch.sqrt(torch.tensor(K.size(self.col_dim), dtype=torch.float32))  # Skala untuk stabilitas

        # Apakah ada mask? Mask digunakan untuk mengabaikan beberapa posisi dalam perhatian
        if mask is not None:
            # Menggunakan mask untuk menutupi (masking) beberapa nilai dalam similarity
            # Misalnya, untuk padding atau posisi yang tidak diinginkan, kita set hasil similarity-nya ke nilai sangat besar
            scaled_sims = scaled_sims.masked_fill(mask, value=1e9)

        # Menghitung skor perhatian dengan Softmax untuk normalisasi
        attention_percentages = F.softmax(scaled_sims, dim=-1)  # Softmax pada dimensi terakhir (seq_len)

        # Menghitung hasil perhatian dengan mengalikan skor perhatian dengan V (Value)
        attention_score = torch.matmul(attention_percentages, V)

        return attention_score

class DecoderOnlyTransformer(L.LightningModule):
    def __init__(self, num_tokens=4, d_model=2, max_len=6):
        super().__init__()

        # Step 1: Embedding layer
        self.embedding = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)

        # Step 2: Positional encoding
        self.position_encoding = PositionEncoding(d_model=d_model, max_len=max_len)

        # Step 3: Attention layer
        self.attention = Attention(d_model=d_model)

        # Step 4: Fully connected layer
        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

        # Step 5: Loss function
        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_id):
        # Step 1: Embedding the input tokens
        word_embeddings = self.embedding(token_id)

        # Step 2: Add positional encoding to the embeddings
        # Slice the positional encoding based on the sequence length of the input (word_embeddings.size(1))
        position_encoded = word_embeddings + self.position_encoding.pe[:word_embeddings.size(1), :]

        # Step 3: Create mask (upper triangular matrix for look-ahead masking)
        seq_length = token_id.size(1)  # panjang urutan
        mask = torch.triu(torch.ones((seq_length, seq_length)), diagonal=1)
        mask = mask == 1  # Membuat mask dengan nilai 1 di atas diagonal (untuk "look-ahead" masking)

        # Step 4: Perform self-attention
        self_attention_values = self.attention(position_encoded, position_encoded, position_encoded, mask)

        # Step 5: Add residual connection
        residual_connection_values = position_encoded + self_attention_values

        # Step 6: Apply the final fully connected layer
        fc_layer_output = self.fc_layer(residual_connection_values)

        return fc_layer_output

    def configure_optimizers(self):
        """Training Optimization"""
        return Adam(self.parameters(), lr=0.1)

    def training_step(self, batch, batch_index):
        """Training Epoch"""
        input_tokens, label = batch
        output = self.forward(input_tokens)
        loss = self.loss(output.view(-1, output.size(-1)), label.view(-1))
        return loss

## predict
---

In [ ]:
# arsitekture model
model = DecoderOnlyTransformer(num_tokens=len(token_to_id),
                               d_model=2, max_len=6)
model

In [ ]:
# Membuat dataset yang lebih beragam dengan kalimat yang lebih kompleks
class SimpleDataset(Dataset):
    def __init__(self, num_samples, seq_length, num_tokens, token_to_id):
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.num_tokens = num_tokens
        self.token_to_id = token_to_id

        # Contoh kalimat yang lebih kompleks
        sentences = [
            ['what', 'is', 'monkey', '<EOS>'],
            ['monkey', 'is','monkey', '<EOS>'],
            ['what', 'is', 'awesome', '<EOS>'],
            ['what', 'monkey', 'awesome', '<EOS>'],
            ['monkey', 'is', 'awesome', '<EOS>'],
            ['awesome', 'is', 'monkey', '<EOS>']
        ]

        self.data = []
        self.labels = []

        for _ in range(num_samples):
            sentence = sentences[_ % len(sentences)]  # Rotasi kalimat untuk variasi
            input_tokens = [self.token_to_id[word] for word in sentence]
            target_tokens = input_tokens[1:] + [self.token_to_id['<EOS>']]  # Label adalah input shift ke kanan

            self.data.append(input_tokens)
            self.labels.append(target_tokens)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor(self.labels[idx])

# Menggunakan dataset baru yang lebih beragam
train_dataset = SimpleDataset(num_samples=1000, seq_length=5, num_tokens=len(token_to_id), token_to_id=token_to_id)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Melihat hasil dataset
for batch_data, batch_labels in train_dataloader:
    print('Batch Data:', batch_data)
    print('Batch Labels:', batch_labels)

In [ ]:
# Setup Trainer
trainer = L.Trainer(max_epochs=30)

# Melatih model
trainer.fit(model, train_dataloader)

In [ ]:
# Inisialisasi input
model_input = torch.tensor([token_to_id['what'],
                            token_to_id['monkey'],
                            token_to_id['is'],
                            token_to_id['<EOS>']])

model_input = model_input.unsqueeze(0)  # Menambahkan dimensi batch (shape: [1, seq_len])

# Tentukan panjang input
input_length = model_input.size(dim=1)

# Prediksi pertama
predictions = model(model_input)  # Mendapatkan output dari model
predicted_id = torch.argmax(predictions[:, -1, :], dim=-1)  # Mengambil prediksi untuk token terakhir
predicted_ids = predicted_id.unsqueeze(0)  # Membuatnya menjadi tensor dengan dimensi yang benar

# Tentukan panjang maksimal output
max_length = 6

# Mulai proses prediksi bertahap
for _ in range(input_length, max_length):
    if predicted_id == token_to_id['what']:
        break

    # Gabungkan prediksi baru ke dalam input model
    model_input = torch.cat([model_input, predicted_ids[:, -1].unsqueeze(1)], dim=1)  # Menambahkan predicted_id ke input

    # Dapatkan prediksi untuk token berikutnya
    predictions = model(model_input)  # Mendapatkan output dari model
    predicted_id = torch.argmax(predictions[:, -1, :], dim=-1)  # Mengambil token dengan skor tertinggi untuk posisi berikutnya

    # Menambahkan predicted_id ke predicted_ids
    predicted_ids = torch.cat([predicted_ids, predicted_id.unsqueeze(0)], dim=1)

# Menampilkan hasil prediksi
print("Predicted Tokens:")
for id in predicted_ids[0]:  # Mengambil urutan token
    print('\t', id_to_token[id.item()])